1. Keep the map processing paused.
2. Turn off/keep only the required layers in the map
3. Turn off the basemap after checking the projection

# Converting the cad files
One file at a time

In [5]:
import arcpy, os, time

- Create an output gdb to keep the converted cad files
- **Fill the parameters below**

In [6]:
# Parameters
input_cad_file = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\DATA\CAD\XREFS\CP1057-GHD-003-XREF-00004_UG_Services.dwg"
spatial_reference = 7855
output_gdb = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\WORKING\NR\Demo\Demo.gdb"

After the tool runs check whether the **cad files are falling on the right place**

In [3]:
# Define the projection
arcpy.DefineProjection_management(input_cad_file, spatial_reference)

<Result 'Z:\\Projects\\VIC\\VIC_HAS\\DATA\\CAD\\XREFS\\CP1057-GHD-003-XREF-00004_UG_Services.dwg'>

In [8]:
# Convert the cad files
# Set the workspace environment to cad file
arcpy.env.workspace = input_cad_file

# List all CAD feature layers from the input CAD file
cad_layers = arcpy.ListFeatureClasses("*")

#Timing
start_time = time.time()
# Convert the files
for cad_layer in cad_layers:
    # Output Layer name
    output_fc_name = os.path.basename(cad_layer)

    # Use FeatureClassToFeatureClass tool to convert each layer
    arcpy.conversion.FeatureClassToFeatureClass(
        in_features=cad_layer,
        out_path=output_gdb,
        out_name=output_fc_name
    )

# Timing
end_time = time.time()
elapsed_time = end_time-start_time
print(f"Execution time: {elapsed_time}")

Execution time: 2068.674816131592


## Only for bulk conversion of cad files

Assuming all the cad files to be converted are kept in a single folder.

Change the input parameters "input_folder", "output_folder", "spatial_reference" while running.

Creates the database automatically.

**Outputs of the geoprocessing are not added into the map to save processing time.**

In [13]:
# Bulk conversion
# Module import
import arcpy, os, time

# Input parameters
input_folder = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\DATA\CAD\030 Construction_staging" # CHANGE AS REQUIRED
output_folder = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\WORKING\NR\Demo\Bulk\030 Construction_staging" # CHANGE AS REQUIRED
spatial_reference = 7855 # CHANGE AS REQUIRED

# Disable adding output to the map
arcpy.env.addOutputsToMap = False

# Record
conversion_time = {}

# Loop through all .dwg files in the folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".dwg"):
        
        # Define file paths
        input_cad_file = os.path.join(input_folder, file_name)
        cad_base_name = os.path.splitext(file_name)[0]
        output_gdb = os.path.join(output_folder, f"{cad_base_name}.gdb")
        size = os.path.getsize(input_cad_file)
        
        print(f"Working on the cad file: {cad_base_name}")
        
        # Create a geodatabase for this CAD file if it doesn't exist
        if not arcpy.Exists(output_gdb):
            print("Creating the geodatabase")
            arcpy.management.CreateFileGDB(output_folder, f"{cad_base_name}")
        
        # Define the projection
        print(f"Defining projection")
        arcpy.DefineProjection_management(input_cad_file, spatial_reference)

        # Set the workspace environment to the current CAD file
        arcpy.env.workspace = input_cad_file
        
        # List all CAD feature layers in the input CAD file
        cad_layers = arcpy.ListFeatureClasses("*")
        
        # Start stopwatch
        start_time = time.time()
        
        print(f"Convertion started for all the layers inside the drawing file")
        # Convert each layer
        for cad_layer in cad_layers:
            # Define output feature class name
            output_fc_name = os.path.basename(cad_layer)

            # Perform the conversion
            arcpy.conversion.FeatureClassToFeatureClass(
                in_features=cad_layer,
                out_path=output_gdb,
                out_name=output_fc_name
            )
        
        # Stop
        end_time = time.time()
        elapsed_time = end_time - start_time
        conversion_time[cad_base_name] = [size, elapsed_time]
        print(f"Conversion completed in {elapsed_time:.2f} seconds.")
        print("--------------------------------------")
        print("\n")

        
print("All files completed")
print("Printing the conversion times to outputted in a csv")
for file in conversion_time:
    print(f"{file},{conversion_time[file][0]},{conversion_time[file][1]}")

Working on the cad file: CP1057-GHD-030-DWG-39001_CAD
Creating the geodatabase
Defining projection
Convertion started for all the layers inside the drawing file
Conversion completed in 222.18 seconds.
--------------------------------------


Working on the cad file: CP1057-GHD-030-DWG-39002_CAD
Creating the geodatabase
Defining projection
Convertion started for all the layers inside the drawing file
Conversion completed in 226.69 seconds.
--------------------------------------


Working on the cad file: CP1057-GHD-030-DWG-39003_CAD
Creating the geodatabase
Defining projection
Convertion started for all the layers inside the drawing file
Conversion completed in 219.33 seconds.
--------------------------------------


Working on the cad file: CP1057-GHD-030-DWG-39004_CAD
Creating the geodatabase
Defining projection
Convertion started for all the layers inside the drawing file
Conversion completed in 221.62 seconds.
--------------------------------------


Working on the cad file: CP1057-

Run the script **to extract the layer properties** to csv from the autocad

Add the converted files into the map

# Update symbology

In [9]:
import csv

In [10]:
# Load CSV into a dictionary (layer name -> (R, G, B)) ---
csv_path = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\WORKING\NR\Demo\output_layers_colors\output_layers_colors.csv"

In [11]:
color_dict = {}

with open(csv_path, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        color_dict[row["Layer Name"]] = (int(row["Red"]), int(row["Green"]), int(row["Blue"]))

Check the first five entries of the csv

In [12]:
count = 0
for key, value in color_dict.items():
    if count < 5:
        print(f"{key}: {value}")
        count += 1
    else:
        break

0: (255, 255, 255)
General-Notes-Disclaimer: (173, 173, 173)
General-North: (173, 173, 173)
Civil-APAM Boundary: (255, 255, 0)
Services-ARP: (255, 255, 0)


In [13]:
# --- 2. Reference your project and map ---
aprx = arcpy.mp.ArcGISProject("CURRENT")  # "CURRENT" is used for scripts run within Pro
map_obj = aprx.activeMap  # Or aprx.listMaps("Map")[0] for the first map

In [14]:
print(map_obj.name)
# print(dir(map_obj))

Map


The following feature layer renderers are currently supported in arcpy.mp:

1. SimpleRenderer
2. GraduatedColorsRenderer
3. GraduatedSymbolsRenderer
4. UnclassedColorsRenderer
5. **UniqueValueRenderer**

## Unique value renderer
1. If you change your current renderer to the UniqueValueRenderer, you first have to set the **appropriate fields** property value.
2. The property is **plural** because you can build a set of unique values based on multiple fields.
3. Therefore, the fields property takes a list, even if there is only one field being used.
4. Once you apply your fields, the renderer will automatically generate all the unique values.
5. If, for whatever reason, new values are added to the layer, you will need to reset the renderer so that all the values are added again. 
6. You can do this by changing the fields property, or you can use the addValues method.
7. You need to understand **how unique values are managed** in the renderer so you can navigate through the class structure to change individual items and their values.
8. The **groups property returns a list of ItemGroup** objects.
9. Each **ItemGroup represents a category of items that each have their own heading**.
10. By default, there is **one ItemGroup**.
11. The **items property** returns a **list of Item objects**.
12. Once you have an item, you can change properties such as label and description. You can also **modify the symbol for each item**.
13. The reason the values property is plural and returns a list is because you can also group items in an ItemGroup. Therefore, each item could have **multiple values**.
14. The following is one way to outline the class structure:
        groups—Returns a list of ItemGroup objects
            1. heading
            2. items—Returns a list of Item objects
                1. description
                2. label
                3. symbol
                4. values
15. You can also **add and remove values** from the renderer.
16. Each **unique value** is actually an **item** in the **object model** and **each item** is associated within an **ItemGroup**.
17. To addValues or removeValues, you need to work with a **Python dictionary**.
18. The dictionary key is the name of the group heading, and its value is a list of values based on the fields property.
19. There is also a **listMissingValues** method that allows you to determine what values are currently missing from the renderer.
20. Values would only be missing if you intentionally called removeValues, because—by default—the renderer automatically creates all unique values when the fields property is set.
21. **If your objective is to create new group headings to organize your unique value items, you must first call removeValues to remove the items, and then use addValues to create a new group with a new heading label**.

In [15]:
# --- 3. Loop through feature layers ---
for lyr in map_obj.listLayers():
    if not lyr.isFeatureLayer:
        continue
    
    desc = arcpy.Describe(lyr)
    if desc.shapeType not in ["Polygon", "Polyline", "Point"]: #Skips the multipatch and annotation geometry
        continue

    fields = [f.name for f in arcpy.ListFields(lyr)]
    if "Layer" not in fields:
        continue

    # --- 4. Setup unique value renderer based on "Layer" field ---
    sym = lyr.symbology

    if hasattr(sym, "renderer"):
        sym.updateRenderer('UniqueValueRenderer')
        sym.renderer.fields = ['Layer']

        # Set each value's color based on color_dict
        for group in sym.renderer.groups:
            for item in group.items:
                lyr_name = item.values[0][0]
                if lyr_name in color_dict:
                    r, g, b = color_dict[lyr_name]
                    symbol = item.symbol
                    # Set color for simple symbols (polygon, line, point)
                    symbol.color = {'RGB': [r, g, b, 100]}  # 100 = full opacity
                    item.symbol = symbol
                else:
                    print("Layer not in color dictionary")

        lyr.symbology = sym  # Apply symbology back to the layer

aprx.save()
print("Symbology updated.")

Symbology updated.


# Publish Layers

## FeaturSharingDraft

### Share a web feature layer
- Share "web feature layers" 

In [9]:
import arcpy

# Current arcgis project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Define the map to publish (update with the name of your map in ArcGIS Pro)
map_name = "Map"
map_to_publish = aprx.listMaps(map_name)[0]

# Define output service parameters
service_name = "Airfield Pavement Markings"  # Set a name for the published service
output_server_folder = "Airfield Pavement Markings"  # Define the folder on the server or portal (set None for root)

# Get the active portal
active_portal = arcpy.GetActivePortalURL()  # Make sure you're logged in!
print(f"Active Portal: {active_portal}")

# Define the service draft
sddraft_path = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\WORKING\NR\Service Definition\service_definition.sddraft"
sd_path = r"\\cgaueprodgissa01.cimic.group\gis-share\Projects\VIC\VIC_HAS\WORKING\NR\Service Definition\service_definition.sd"

# Create a web layer sharing draft
sharing_draft = map_to_publish.getWebLayerSharingDraft("HOSTING_SERVER", "WEBMAP", service_name)
sharing_draft.portalFolder = output_server_folder  # Default is same as server folder assign the portal folder

# Export to a Service Definition Draft file
sharing_draft.exportToSDDraft(sddraft_path)

# Stage the service (create .sd file)
arcpy.StageService_server(sddraft_path, sd_path)

# Publish the service to the active portal
arcpy.UploadServiceDefinition_server(sd_path, "My Hosted Services")

print("Web map successfully published!")

Active Portal: https://gis-dev.cimic.com.au/portal/


ValueError: Incompatible server_type: FEDERATED_SERVER and service_type: WEBMAP

# Delete extra resources